# 1. Particles: First Data Inspection

## Simulation Overview

In [20]:
using Mera
#info = getinfo(1, "../../testing/simulations/manu_stable_2019");
info = getinfo(400, "../../testing/simulations/manu_sim_sf_L14");

[Mera]: 2020-02-12T20:40:36.118

Code: RAMSES
output [400] summary:
mtime: 2018-09-05T09:51:55.041
ctime: 2019-11-01T17:35:21.051
simulation time: 594.98 [Myr]
boxlen: 48.0 [kpc]
ncpu: 2048
ndim: 3
-------------------------------------------------------
amr:           true
level(s): 6 - 14 --> cellsize(s): 750.0 [pc] - 2.93 [pc]
-------------------------------------------------------
hydro:         true
hydro-variables:  7  --> (:rho, :vx, :vy, :vz, :p, :var6, :var7)
hydro-descriptor: (:density, :velocity_x, :velocity_y, :velocity_z, :thermal_pressure, :passive_scalar_1, :passive_scalar_2)
γ: 1.6667
-------------------------------------------------------
gravity:       true
gravity-variables: (:epot, :ax, :ay, :az)
-------------------------------------------------------
particles:     true
- Npart:    5.091500e+05 
- Nstars:   5.066030e+05 
- Ndm:      2.547000e+03 
particle variables: (:vx, :vy, :vz, :mass, :birth)
-------------------------------------------------------
clumps:       

A short overview of the loaded particle properties is printed:
- existence of particle files
- the predefined variables
- the number of particles for each id/family (if exist)
- the variable names from the descriptor file (if exist)


The functions in **Mera** "know" the predefined particle variable names: 
- From >= ramses-version-2018: :vx, :vy, :vz, :mass, :family, :tag, :birth, :metals :var9,.... 
- For  =< ramses-version-2017: :vx, :vy, :vz, :mass, :birth, :var6, :var7,.... 
- Currently, the following variables are loaded by default (if exist): :level, :x, :y, :z, :id, :family, :tag.
- The cpu number associated with the particles can be loaded with the variable names: :cpu or :varn1
- In a future version the variable names from the particle descriptor can be used by setting the field info.descriptor.useparticles = true . 

Get an overview of the loaded particle properties:

In [21]:
viewfields(info.part_info)


[Mera]: Particle overview
eta_sn	= 0.0
age_sn	= 0.6706464407596582
f_w	= 0.0
Npart	= 509150
Ndm	= 2547
Nstars	= 506603
Nsinks	= 0
Ncloud	= 0
Ndebris	= 0
Nother	= 0
Nundefined	= 0
other_tracer1	= 0
debris_tracer	= 0
cloud_tracer	= 0
star_tracer	= 0
other_tracer2	= 0
gas_tracer	= 0



## Load AMR/Particle Data

Read the AMR and the Particle data from all files of the full box with all existing variables and particle positions:

In [22]:
particles = getparticles(info);

[Mera]: Get particle data: 2020-02-12T20:40:36.139

Key vars=(:level, :x, :y, :z, :id)
Using var(s)=(1, 2, 3, 4, 5) = (:vx, :vy, :vz, :mass, :birth) 

domain:
xmin::xmax: 0.0 :: 1.0  	==> 0.0 [kpc] :: 48.0 [kpc]
ymin::ymax: 0.0 :: 1.0  	==> 0.0 [kpc] :: 48.0 [kpc]
zmin::zmax: 0.0 :: 1.0  	==> 0.0 [kpc] :: 48.0 [kpc]



Reading data...100%|████████████████████████████████████| Time: 0:00:03


Found 5.089390e+05 particles
Memory used for data table :34.947275161743164 MB
-------------------------------------------------------



The memory consumption of the data table is printed at the end. We provide a function which gives the possibility to print the used memory of any object: 

In [23]:
usedmemory(particles);

Memory used: 35.211 MB


The assigned object is now of type `PartDataType`:

In [24]:
typeof(particles)

PartDataType

It is a sub-type of ContainMassDataSetType

In [25]:
supertype( ContainMassDataSetType )

DataSetType

ContainMassDataSetType is a sub-type of to the super-type DataSetType

In [26]:
supertype( PartDataType )

HydroPartType

The data is stored in a **JuliaDB** table and the user selected particle variables and parameters are assigned to fields:

In [27]:
viewfields(particles)


data ==> JuliaDB table: (:level, :x, :y, :z, :id, :vx, :vy, :vz, :mass, :birth)

info ==> subfields: (:output, :path, :fnames, :simcode, :mtime, :ctime, :ncpu, :ndim, :levelmin, :levelmax, :boxlen, :time, :aexp, :H0, :omega_m, :omega_l, :omega_k, :omega_b, :unit_l, :unit_d, :unit_m, :unit_v, :unit_t, :gamma, :hydro, :nvarh, :nvarp, :variable_list, :gravity_variable_list, :particles_variable_list, :clumps_variable_list, :sinks_variable_list, :descriptor, :amr, :gravity, :particles, :clumps, :sinks, :rt, :namelist, :namelist_content, :headerfile, :makefile, :files_content, :timerfile, :compilationfile, :patchfile, :Narraysize, :scale, :grid_info, :part_info, :compilation, :constants)

lmin	= 6
lmax	= 14
boxlen	= 48.0
ranges	= [0.0, 1.0, 0.0, 1.0, 0.0, 1.0]
selected_partvars	= Symbol[:level, :x, :y, :z, :id, :vx, :vy, :vz, :mass, :birth]

scale ==> subfields: (:Mpc, :kpc, :pc, :mpc, :ly, :Au, :km, :m, :cm, :mm, :μm, :Msol_pc3, :g_cm3, :Msol_pc2, :g_cm2, :Gyr, :Myr, :yr, :s, :ms, :Msol, :

For convenience, all the fields from the info-object above (InfoType) are now also accessible from the object with "particles.info" and the scaling relations from code to cgs units in "particles.scale".

Print the fields of an object (composite type) in a simple list:

In [28]:
propertynames(particles)

(:data, :info, :lmin, :lmax, :boxlen, :ranges, :selected_partvars, :used_descriptors, :scale)

## Overview of AMR/Particles
Get an overview of the AMR structure associated with the object `particles` (PartDataType). The printed information is stored into the object `overview_amr` as a **JuliaDB** table (code units)  and can be used for further calculations:

In [29]:
amr_overview = amroverview(particles)

Counting...


Table with 9 rows, 2 columns:
level  particles
────────────────
6      2867
7      11573
8      56209
9      118156
10     115506
11     77303
12     49300
13     31859
14     46166

Get some overview of the data that is associated with the object `particles`. The calculated information can be accessed from the object `data_overview` (here) in code units for further calculations:

In [30]:
data_overview = dataoverview(particles)

Table with 9 rows, 19 columns:
Columns:
#   colname    type
───────────────────
1   level      Any
2   x_min      Any
3   x_max      Any
4   y_min      Any
5   y_max      Any
6   z_min      Any
7   z_max      Any
8   id_min     Any
9   id_max     Any
10  vx_min     Any
11  vx_max     Any
12  vy_min     Any
13  vy_max     Any
14  vz_min     Any
15  vz_max     Any
16  mass_min   Any
17  mass_max   Any
18  birth_min  Any
19  birth_max  Any

If the number of columns is relatively long, the table is typically represented by an overview. To access certain columns, use the `select` function. The representation ":birth_max" is called a quoted Symbol ([see in Julia documentation](https://docs.julialang.org/en/v1/manual/metaprogramming/#Symbols-1)):

In [31]:
using JuliaDB

In [32]:
select(data_overview, (:level,:mass_min, :mass_max, :birth_min, :birth_max ) )

Table with 9 rows, 5 columns:
level  mass_min    mass_max    birth_min  birth_max
───────────────────────────────────────────────────
6      1.13606e-5  1.13606e-5  4.54194    38.2611
7      1.13606e-5  1.13606e-5  4.35262    38.6822
8      1.13606e-5  2.27212e-5  4.30782    39.2733
9      1.13606e-5  2.27212e-5  4.39811    39.397
10     1.13606e-5  2.27212e-5  4.29246    39.6783
11     1.13606e-5  2.27212e-5  4.36385    39.744
12     1.13606e-5  2.27212e-5  4.51187    39.8358
13     4.99231e-6  2.27212e-5  4.47027    39.883
14     4.91416e-6  2.27212e-5  5.2582     39.902

Get an array from the column ":birth" in `data_overview` and scale it to the units `Myr`. The order of the calculated data is consistent with the table above:

In [33]:
column(data_overview, :birth_min) * info.scale.Myr 

9-element Array{Float64,1}:
 67.72475450854526
 64.901905404619  
 64.23377578948651
 65.58011000778922
 64.00484282690913
 65.06931266062217
 67.27645257301232
 66.6561171130916 
 78.40495776518468

Or simply convert the `birth_max` data in the table to `Myr` units by manipulating the column:

In [34]:
data_overview = transform(data_overview, :birth_max => :birth_max => value->value * info.scale.Myr);

In [35]:
select(data_overview, (:level,:mass_min, :mass_max, :birth_min, :birth_max ) )

Table with 9 rows, 5 columns:
level  mass_min    mass_max    birth_min  birth_max
───────────────────────────────────────────────────
6      1.13606e-5  1.13606e-5  4.54194    570.511
7      1.13606e-5  1.13606e-5  4.35262    576.79
8      1.13606e-5  2.27212e-5  4.30782    585.604
9      1.13606e-5  2.27212e-5  4.39811    587.448
10     1.13606e-5  2.27212e-5  4.29246    591.643
11     1.13606e-5  2.27212e-5  4.36385    592.623
12     1.13606e-5  2.27212e-5  4.51187    593.991
13     4.99231e-6  2.27212e-5  4.47027    594.694
14     4.91416e-6  2.27212e-5  5.2582     594.977

## Data inspection
The data is associated with the field `particles.data` as a **JuliaDB** table (code units). 
Each row corresponds to a particle and each column to a property which makes it easy to find, filter, map, aggregate, group the data, etc.
More information can be found in the **Mera** tutorials or in: [JuliaDB API Reference](http://juliadb.org/latest/api/)


### Table View

The particle positions x,y,z are given in code units and used in many functions of **MERA** and should not be modified.

In [36]:
particles.data

Table with 508939 rows, 10 columns:
Columns:
#   colname  type
────────────────────
1   level    Int32
2   x        Float64
3   y        Float64
4   z        Float64
5   id       Int32
6   vx       Float64
7   vy       Float64
8   vz       Float64
9   mass     Float64
10  birth    Float64

A more detailed view into the data:

In [37]:
select(particles.data, (:level,:x, :y, :z, :birth) )

Table with 508939 rows, 5 columns:
level  x           y        z        birth
────────────────────────────────────────────
6      0.00462947  22.3885  24.571   32.0735
6      0.109066    22.3782  21.5844  19.8963
6      0.238211    28.7537  24.8191  24.9471
6      0.271366    22.7512  31.5681  20.9888
6      0.312574    16.2385  23.7591  23.0935
6      0.314957    28.2084  30.966   31.6911
6      0.328337    4.59858  23.5001  30.3666
6      0.420712    27.6688  26.5735  18.9512
6      0.509144    33.1737  23.9789  24.3613
6      0.565516    25.9409  26.0579  32.7551
6      0.587289    9.60231  23.8477  29.6981
6      0.592878    25.5519  21.3079  15.9204
⋮
14     37.6271     25.857   23.8833  36.7754
14     37.6299     25.8403  23.9383  36.0289
14     37.6301     25.8502  23.9361  38.7225
14     37.6326     25.8544  23.9383  36.3547
14     37.6528     25.8898  23.9928  38.2109
14     37.6643     25.9061  23.9945  39.49
14     37.6813     25.8743  23.9789  36.6981
14     37.7207     25.